<a href="https://colab.research.google.com/github/sergeymasl/pandas_cource/blob/main/Renaming_and_Combining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Renaming and Combining**

## **Введение**

В этот уроке вы изучите методы по изменению названий столбцов или индексов.

Также вы изучите как соединить несколько DataFrame или несколько Series

In [2]:
# импорт библиотеки
import pandas as pd
# настройка отображения
pd.set_option('max_rows', 5)
# загрузка датасета для дальнейшей работы
movies_director = pd.read_csv("https://drive.google.com/uc?export=download&id=1vnRsmnTDLNC7xfLviXWRe3DxnKIPBS9k", index_col=0, sep = ';')
movies_release = pd.read_csv("https://drive.google.com/uc?export=download&id=1Om_PzTwtWnj6BKXKndGpX3bJH8j2zM26", index_col=0, sep = ';')

Посмотрим на данные с которыми предстоит работать:

In [3]:
movies_director

,movie_name,director
0,Titanic,James Cameron
1,Green Book,Peter Farrelly
...,...,...
9,Hachi: A Dog's Tale,Lasse Hallstrom
10,Police Story,Jackie Chan


In [4]:
movies_release

,movie_name,release_date
0,Sherlock Holmes and Dr. Watson,"March 22, 1980"
1,Psycho,"June 16, 1960"
...,...,...
5,"Lock, Stock and Two Smoking Barrels","August 28, 1998"
6,12 Angry Men,"April 10, 1957"


## **Переименование**

Первый метод, который мы рассмотрим это `rename()`:

`rename(mapper=None, index=None, columns=None, axis=None, copy=True, inplace=False, level=None, errors='ignore')`

- `mapper` - словарь в виде `{'old_name' : 'new_name'}` или функция для переименования столбцов или строк, в зависимости от параметра `axis`
- `index` - словарь в виде `{'old_name_index' : 'new_name_index'}` для переименования индексов 
- `columns` - словарь в виде `{'old_name_column' : 'new_name_column'}` для переименования индексов
- `axis` - `{0 or ‘index’, 1 or ‘columns’}` - выбор оси, при  исползовании `mapper` 
- `copy` - параметр, отвечающий за создание копии исходных данных *(автор не нашел вариантов использования этого параметра)*
- `inplace` - параметр, отвечающий за изменение исходного массива
- `level` - уровень в мульти-индексе в котором необходимо произвести переименование
- `errors` - `{‘ignore’, ‘raise’}` - метод отвечающий за действия в случае обнаружения имени переименовываемого столбца/индекса которого нет в DataFrame.
> `raise` - если в параметре `mapper` или `index` или `columns` будет передано в виде старое имя которого нет в DataFrame, то будет выведено исключение 
>
> `ignore` - если будет передано старое имя, которого нет, метод продолжит работу, проигнорируя исключение


**Вы можете использовать либо  сочетание `mapper` и `axis`, либо параметры `axis` и/или `columns`**

**Пример:**

Переименуем столбцы c помощю функции и индексы с помощью словаря:

In [5]:
movies_director.rename(columns = str.upper, index = {0 : 'first_movie', 1 : 'second_movie'})

,MOVIE_NAME,DIRECTOR
first_movie,Titanic,James Cameron
second_movie,Green Book,Peter Farrelly
...,...,...
9,Hachi: A Dog's Tale,Lasse Hallstrom
10,Police Story,Jackie Chan


И посмотрим отработку параметра `errors`:

In [ ]:
try:
  # имя 'producer' нет в названии столбца
  # errors по умолчанию
  movies_director.rename(columns = {'producer' : 'продюсер'})
  print("Метод rename c параметром errors \033[1mпо умолчанию\033[0m отработал и не вызвал исключение\n")
  # errors - 'raise'
  movies_director.rename(columns = {'producer' : 'продюсер'}, errors = 'raise')
except Exception as ex:
  print("Метод rename c параметром errors - \033[1m'raise'\033[0m вызвал исключение:")
  print(f'Тип исключения: \033[1m{type(ex)}\033[0m')
  print(f'Описание исключения: \033[1m{ex}\033[0m')

Метод rename c параметром errors по умолчанию отработал и не вызвал исключение

Метод rename c параметром errors - 'raise' вызвал исключение:
Тип исключения: <class 'KeyError'>
Описание исключения: "['producer'] not found in axis"


# **Сопоставление**

В процессе работы над данными часто, возникает потребность в сопоставлении или содинении данны из разных таблиц.

Pandas имеет три основных метода сопоставления данных:
- `pd.concat()`
- `pd.join()`
- `pd.merge()`

## `pd.concat()`
Первый метод который мы рассмотрим, это `pd.concat()`, он применяется когда данные хранятся в нескольких DataFrame и/или Series, и их необходимо соединить вместе без каких-либо сопоставлений.
Функция объединяет данные вдоль одной из осей:

![fillna_1.PNG](https://raw.githubusercontent.com/sergeymasl/pandas_cource/main/picture/concat.PNG)

С помощью `pd.concat()` можно соединять как DataFrame, так и Series.

In [30]:
series_1 = movies_release['movie_name']
series_2 = movies_release['movie_name'].map(str.upper)
pd.concat([series_1, series_2], axis = 1)

,movie_name,movie_name
0,Sherlock Holmes and Dr. Watson,SHERLOCK HOLMES AND DR. WATSON
1,Psycho,PSYCHO
...,...,...
5,"Lock, Stock and Two Smoking Barrels","LOCK, STOCK AND TWO SMOKING BARRELS"
6,12 Angry Men,12 ANGRY MEN


Теперь рассмотрим синтаксис и опишем основные параметры:

`pandas.concat(objs, axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=False, copy=True)`

- `objs` - cписок с DataFrame и/или Series `[df1, df2, series1, series2]`
- `axis` - `{0 or ‘index’, 1 or ‘columns’}` - выбор оси, вдоль которой нужно сопоставить DataFrame или Series
- `join` - `bool` - `{‘inner’, ‘outer’}` - способ подстановки данных, при `outer` данные будут подставлены вне зависимости от наличия одинаковых столбцов(axis = 0) или индексов(axis = 1) при `inner` будут сопоставленны столько строки/столбцы имеющие одинаковые индексы (ниже подробнее)
- `ignore_index` - если присвоить параметру `True`, то в объединенном массиве индексы или названия столбцов (в зависимости от выбора оси работы метода) будут проигнорированны и созданы новые от 0 до n-1
- `keys` - в параметр передается список с "ключами" идентифицирующими соединенные фреймы данных, размер списка должен быть равным количеству соединяемых фреймов. При соединении вдоль индексов будет создан мультииндекс первым уровнем которого будуд "ключи", если соединение вдоль столбцов, то будут переименованны столбцы.


**Подробнее о параметре join:**

`df1 = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number'])`

|    | letter   |   number |
|:---|:--------:|:--------:|
|  0 | a        |        1 |
|  1 | b        |        2 |

`df2 = pd.DataFrame([['c', 3, 'cat'], ['d', 4, 'dog']], columns=['letter', 'number', 'animal'])`

|    | letter   |   number | animal   |
|:---|:--------:|:--------:|:--------:|
|  0 | c        |        3 | cat      |
|  1 | d        |        4 | dog      |


`pd.concat([df1, df2], join="outer")`

|    | letter   |   number | animal   |
|:---|:--------:|:--------:|:--------:|
|  0 | a        |        1 | NaN      |
|  1 | b        |        2 | NaN      |
|  0 | c        |        3 | cat      |
|  1 | d        |        4 | dog      |

`pd.concat([df1, df2], join="inner")`

|    | letter   |   number |
|:---|:--------:|:--------:|
|  0 | a        |        1 |
|  1 | b        |        2 |
|  0 | c        |        3 |
|  1 | d        |        4 |


Более подробная информация и примеры использования в [оф.документации](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)


## pd.join()

А теперь прежде чем мы перейдем к изучению метода, рассмотрим само понятие `join`. 

Итак join имеет 4 варианта сопоставления: 
- `left join`
- `right join`
- `outer join`
- `inner join`

**Рассмотрим на примере**:
мы имеем два набора данных, **первый** это набор названий фильмов с указанием режисеров, **второй** это набор названий фильмов с датами их релизов. *Ключами по которым можно идентифицировать, в данном случае фильмы, это, собственно, сами названия фильмов. В этом массиве они уникальны, хотя в жизни все бывает сложнее.* 

Фильмы между этими массивами **могут пересекаться, а могут не пересекаться**.

>**первый массив** `left join` **второй массив** - это когда ко всем строкам **первого массива** присвоят **СОВПАДАЮЩИЕ** строки из **второго массива**

>**первый массив** `right join` **второй массив** - это когда наоборот, всем строкам **второго массива** присвоят **СОВПАДАЮЩИЕ** строки из **перового массива**

>**первый массив** `outer join` **второй массив** - это когда в результат будут выведены **все строки из обоих наборов** вне зависимости от пересечений

>**первый массив** `inner join` **второй массив** - это когда в результат будут выведены **ТОЛЬКО пересекающиеся строки**








[Хорошая статейка про join-ы](https://skillbox.ru/media/code/metodologiya-bem-chto-eto-takoe-i-kak-ona-uproshchaet-zhizn-razrabotchikam/)